In [4]:
from typing import Text, Generator, Tuple, List, Optional, Dict, Set
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import json
import random
import re
import base64
import ast
import copy
import imgkit
import math
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
from tabulate import tabulate
sns.set_theme()

config = imgkit.config(wkhtmltoimage='C:/Program Files/wkhtmltopdf/bin/wkhtmltoimage.exe')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:.2f}'.format

# 0. OpenAI

In [5]:
from openai import OpenAI

client = OpenAI()

In [6]:
def generate_completion(prompt, model='gpt-4o', temp=0.50, n=1, max_tokens=30000, logprobs=False, top_logprobs=5, return_obj=True, verbose=False, num_rows=-1):
    if verbose:
        if num_rows == -1:
            print('Generating completion...')
        else:
            print('[{}] - Generating completion...'.format(num_rows))
    if logprobs:
        chat_completion = client.chat.completions.create(
              model=model,
              messages=[
                    {"role": "user", "content": prompt},
                ],
              temperature=temp,
              n=n,
              logprobs=logprobs,
        )
    else:
        chat_completion = client.chat.completions.create(
              model=model,
              messages=[
                    {"role": "user", "content": prompt},
                ],
              temperature=temp,
              n=n,
        )
        if chat_completion.choices[0].finish_reason == 'length':
            print("Response cut off. Starting a second request... Length: {}".format(chat_completion.usage.completion_tokens))
            prompt += '\n -------- \nYour last response was cut off. Continue exactly where you left.'
            chat_completion_2 = client.chat.completions.create(
              model=model,
              messages=[
                    {"role": "user", "content": prompt},
                      {
                        "role": "assistant",
                        "content": chat_completion.choices[0].message.content
                    }
                ],
              temperature=temp,
              n=n,
        )
            chat_completion.choices[0].message.content += chat_completion_2.choices[0].message.content
            chat_completion.usage.completion_tokens += chat_completion_2.usage.completion_tokens
            chat_completion.usage.prompt_tokens += chat_completion_2.usage.prompt_tokens
            chat_completion.usage.total_tokens += chat_completion_2.usage.total_tokens
    return chat_completion if return_obj else [choice.message.content for choice in chat_completion.choices]

In [7]:
def extract_html(completion):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', completion, re.DOTALL)
    if html_content:
        extracted_text = html_content.group()
        extracted_text = extracted_text.replace('```html','').replace('```','')
        return extracted_text
    else:
        html_content = re.search(r'<html>.*</html>', completion, re.DOTALL)
        if html_content:
            extracted_text = html_content.group()
            extracted_text = extracted_text.replace('```html','').replace('```','')
            return extracted_text
    return ''

def extract_html_cot(completion):
    html_content = re.search(r'<!-- Start -->.*<!-- End -->', completion, re.DOTALL)
    extracted_text = html_content.group()
    extracted_text = extracted_text.replace('```html','').replace('```','')
    return extracted_text

def write_html_to_file_and_image(path, model_name, dataset):
    for index, row in dataset.iterrows():
        file_name = str(index)
        with open(path+file_name+'.html', 'w', encoding='utf-8') as file:
            print(path+file_name+'.html')
            file.write(row[model_name])
        try:
            imgkit.from_string(row[model_name], path+file_name+'.jpg', config=config)
        except:
            pass

def save_results(dataframe, method_name, keep_cols, completion_col, path_csv, path_html, html_extractor=extract_html, is_json=False):
    df_copy = dataframe.copy()
    selected_cols = ['RICO GUI', 'description', completion_col]
    selected_cols.extend(keep_cols)
    df_copy = df_copy[selected_cols]
    df_copy['method'] = method_name
    if is_json:
        df_copy['generated_html'] =  df_copy[completion_col].apply(lambda x: html_extractor(x['choices'][0]['message']['content']))
        df_copy['token_input'] = df_copy[completion_col].apply(lambda x: x['usage']['prompt_tokens'])
        df_copy['token_output'] = df_copy[completion_col].apply(lambda x: x['usage']['completion_tokens'])
    else:
        df_copy['generated_html'] =  df_copy[completion_col].apply(lambda x: html_extractor(x.choices[0].message.content))
        df_copy['token_input'] = df_copy[completion_col].apply(lambda x: x.usage.prompt_tokens)
        df_copy['token_output'] = df_copy[completion_col].apply(lambda x: x.usage.completion_tokens)
    for col in df_copy.columns:
        if "completion" in col:
            df_copy[col] = df_copy[col].to_dict()
    df_copy.to_csv(path_csv + method_name + '.csv', index=False)
    write_html_to_file_and_image(path_html, 'generated_html', df_copy)

# 1. Load Datasets

In [20]:
df_sample_descriptions_test = pd.read_csv('../data/generated/00_dataset_test/data/01_baselines/zs_instruction.csv')
df_sample_descriptions_test['rico_ranking'] = df_sample_descriptions_test['rico_ranking'].apply(ast.literal_eval)
df_sample_descriptions_test['binary_annotation'] = df_sample_descriptions_test['binary_annotation'].apply(ast.literal_eval)
df_sample_descriptions_test.rename(columns={'Descriptions': 'description'}, inplace=True)
df_sample_descriptions_test['filtered_rico_ids'] = df_sample_descriptions_test.apply(lambda x: [], axis=1)

# 2. k-Self-Critique Looping

In [22]:
df_sample_descriptions_with_prompts = df_sample_descriptions_test.copy()

In [23]:
PLACEHOLDER_PROTOTYPE = '{prototype}'
PLACEHOLDER_FEEDBACK = '{feedback}'

In [25]:
FEEDBACK_PROMPT = """Your task is to create feedback on a GUI prototype, that has been generated based on a high-level text description.

### High-level text description:
{summary}

### GUI prototype in the form of HTML and CSS code:
{prototype}

### Your Task
- Provide feedback on the prototype, regarding the following:
    - What features would be also relevant to include into the prototype?
    - How can the implementation of the current present features be improved?
    - How can the design and layout of the overall prototype be improved?
- Do not provide any specific implementations. Solely provide the feedback in text form.
- Focus on HMTL and CSS implementation. 
- IMPORTANT: Avoid feedback, that requires any Java Script implementation."""

In [26]:
REFINEMENT_PROMPT = """Improve a given GUI prototype based on feedback that is also provided.

### High-level text description, the GUI prototype is based on:
{summary}

### Curent implementation of the GUI prototype in the form of HTML and CSS code:
{prototype}

### Feedback, that was created to improve the current prototype implementation:
{feedback}

### Your Task
- Improve the Current GUI prototype, by integrating the provided Feedback into a new GUI prototype. 
- Respond with the revised HTML and CSS Code, without any additional explanation. 
- Provide the HTML and CSS together, do not seperate them.
- IMPORTANT: Avoid adding any java script code to the prototype"""

In [27]:
TEMPERATURE = 0.5
K = 4

In [ ]:
for i in range(0,K):

    # 1. Generate feedback and extract content
    print('Generating feedback_v{}'.format(str(i)))
    df_sample_descriptions_with_prompts['feedback_prompt_v'+str(i)] = df_sample_descriptions_with_prompts.apply(lambda row: FEEDBACK_PROMPT.replace(PLACEHOLDER_SUMMARY, row['description']).replace(PLACEHOLDER_PROTOTYPE, row['prototype_completion_content_v'+str(i)]), axis=1)
    df_sample_descriptions_with_prompts['feedback_completion_v'+str(i)] = df_sample_descriptions_with_prompts.apply(lambda row: generate_completion(row['feedback_prompt_v'+str(i)], model='gpt-4o', temp=TEMPERATURE, logprobs=False, return_obj=True, verbose=True, num_rows=row.name), axis=1)
    df_sample_descriptions_with_prompts['feedback_completion_content_v'+str(i)] = df_sample_descriptions_with_prompts.apply(lambda row: row['feedback_completion_v'+str(i)].choices[0].message.content, axis=1)
    
    # 2. Generate prototype and extract HTML
    print('Implementing prototype_v{}'.format(str(i+1)))
    df_sample_descriptions_with_prompts['prototype_prompt_v'+str(i+1)] = df_sample_descriptions_with_prompts.apply(lambda row: REFINEMENT_PROMPT.replace(PLACEHOLDER_SUMMARY, row['description']).replace(PLACEHOLDER_PROTOTYPE, row['prototype_completion_content_v'+str(i)]).replace(PLACEHOLDER_FEEDBACK, row['feedback_completion_content_v'+str(i)]), axis=1)
    df_sample_descriptions_with_prompts['prototype_completion_v'+str(i+1)] = df_sample_descriptions_with_prompts.apply(lambda row: generate_completion(row['prototype_prompt_v'+str(i+1)], model='gpt-4o', temp=TEMPERATURE, logprobs=False, return_obj=True, verbose=True, num_rows=row.name), axis=1)
    df_sample_descriptions_with_prompts['prototype_completion_content_v'+str(i+1)] = df_sample_descriptions_with_prompts.apply(lambda row: row['prototype_completion_v'+str(i+1)].choices[0].message.content, axis=1)
    
    # 3. Generate keep columns to save
    feedback_keep_cols = ['feedback_completion_v'+str(j+1) for j in range(-1,i)]
    prototype_keep_cols = ['prototype_completion_v'+str(x+1) for x in range(-1,i-1)]
    keep_cols = feedback_keep_cols
    keep_cols.extend(prototype_keep_cols)
    try:
        keep_cols.remove('prototype_completion_v0')
    except:
        pass
    print(keep_cols)
    
    # 4. Save results
    save_results(df_sample_descriptions_with_prompts, 'prototype_v'+str(i+1), keep_cols, 'prototype_completion_v'+str(i+1), '../data/generated/00_dataset_test/data/04_self_critique/base_zs_instruction/', '../data/generated/00_dataset_test/guis/04_self_critique/base_zs_instruction/prototype_v'+str(i+1)+'/', is_json=False)